In [3]:
import requests 

# API 엔드포인트 URL 및 매개 변수 설정함
url = "https://openapi.its.go.kr:9443/cctvInfo"
params = {
    "apiKey": "848b2f28c3184279a1935e17289e0009",  # 실제 API 키
    "type": "all",
    "cctvType": "1",
    "minX": "126.800000",
    "maxX": "127.890000",
    "minY": "34.900000",
    "maxY": "35.100000",
    "getType": "xml"
}

# GET 요청 보내기
response = requests.get(url, params=params)

# 응답 코드 확인
print("Response code:", response.status_code)

# 응답 내용 출력
if response.status_code == 200:
    print(response.text)
else:
    print("Error:", response.text)


Response code: 200
<?xml version="1.0" encoding="UTF-8"?>
<response>
	<coordtype>1</coordtype>
	<datacount>77</datacount>
	<data>
		<roadsectionid/>
		<filecreatetime/>
		<cctvtype>1</cctvtype>
		<cctvurl>http://cctvsec.ktict.co.kr/138/UQikOtaEpzU6qv/yYy87sPmQ7HrBVet8TT5GGnSbpQTUiorDy8hq3pcV+RBibhp5772jCFfa9R55OfPDamRWB4egghfAQLZW6C9cKi89GEY=</cctvurl>
		<cctvresolution/>
		<coordy>34.9977531433105</coordy>
		<cctvformat>HLS</cctvformat>
		<cctvname>[남해선] 죽평</cctvname>
		<coordx>127.492691040039</coordx>
	</data>
	<data>
		<roadsectionid/>
		<filecreatetime/>
		<cctvtype>1</cctvtype>
		<cctvurl>http://cctvsec.ktict.co.kr/139//tnub+esIOQoY0aBGihJV0GbUt1cC2JfQmuoxAS52/aOGCUZKnn0waocdUFyRU4M214FGTaxfJa4Q42CLLmeo7cySB4Ce2iSAZD3ZZcYJbg=</cctvurl>
		<cctvresolution/>
		<coordy>34.9904441833496</coordy>
		<cctvformat>HLS</cctvformat>
		<cctvname>[남해선] 지본교</cctvname>
		<coordx>127.505096435546</coordx>
	</data>
	<data>
		<roadsectionid/>
		<filecreatetime/>
		<cctvtype>1</cctvtype>
		<cctvurl>

In [4]:
import requests
import cv2

# API 엔드포인트 URL 및 매개 변수 설정
api_url = "https://openapi.its.go.kr:9443/cctvInfo"
params = {
    "apiKey": "848b2f28c3184279a1935e17289e0009",  # 실제 API 키
    "type": "all",
    "cctvType": "1",
    "minX": "126.800000",
    "maxX": "127.890000",
    "minY": "34.900000",
    "maxY": "35.100000",
    "getType": "xml"
}

# GET 요청 보내기
response = requests.get(api_url, params=params)

# 응답 코드 확인
print("Response code:", response.status_code)

if response.status_code == 200:
    # XML 파싱
    from xml.etree import ElementTree as ET
    root = ET.fromstring(response.text)

    # CCTV URL 추출
    cctv_data = root.findall(".//data")
    for data in cctv_data:
        cctv_url = data.find("cctvurl").text

        # OpenCV로 비디오 스트리밍 실행
        cap = cv2.VideoCapture(cctv_url)

        while cap.isOpened():
            ret, frame = cap.read()

            if ret:
                cv2.imshow("CCTV Streaming", frame)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            else:
                break

        cap.release()
        cv2.destroyAllWindows()
else:
    print("Failed to fetch CCTV data")


Response code: 200


In [32]:
import requests
import cv2
import torch
from xml.etree import ElementTree as ET
from ultralytics import YOLO

# YOLO 모델 불러오기
#model = torch.load('yolov8n.pt')
model = YOLO('..\\..\\data\\yolov8\\yolov8n.pt')

# API 엔드포인트 URL 및 매개 변수 설정
api_url = "https://openapi.its.go.kr:9443/cctvInfo"
params = {
    "apiKey": "848b2f28c3184279a1935e17289e0009",  # 실제 API 키
    "type": "all",
    "cctvType": "1",
    # "minX": "126.800000",   #최소 경도 영역
    # "maxX": "127.890000",   #최대 경도 영역
    # "minY": "34.900000",    #최소 위도 영역
    # "maxY": "35.100000",    #최대 위도 영역
    # "getType": "xml"        #출력 결과 형식

    "minX": "126.800000",   #최소 경도 영역
    "maxX": "127.000000",   #최대 경도 영역
    "minY": "37.900000",    #최소 위도 영역
    "maxY": "38.400000",    #최대 위도 영역
    "getType": "xml"        #출력 결과 형식
}

# GET 요청 보내기
response = requests.get(api_url, params=params)

# 응답 코드 확인
print("Response code:", response.status_code)

if response.status_code == 200:
    # XML 파싱
    root = ET.fromstring(response.text)

    # CCTV URL 추출
    cctv_data = root.findall(".//data")
    for data in cctv_data:
        cctv_url = data.find("cctvurl").text

        # OpenCV로 비디오 스트리밍 실행
        cap = cv2.VideoCapture(cctv_url)

        while cap.isOpened():
            ret, frame = cap.read()

            if ret:
                # YOLO로 차량 감지
                results = model(frame)
                #results.render()  # 결과를 프레임에 렌더링
                #cv2.imshow("CCTV Streaming", frame)

                # Visualize the results on the frame
                annotated_frame = results[0].plot()
                # Display the annotated frame
                cv2.imshow("YOLOv8 Inference", annotated_frame)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            else:
                break

        cap.release()
        cv2.destroyAllWindows()
else:
    print("Failed to fetch CCTV data")


Response code: 200



0: 448x640 12 cars, 1 truck, 46.6ms
Speed: 5.5ms preprocess, 46.6ms inference, 0.8ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 11 cars, 1 truck, 53.1ms
Speed: 0.8ms preprocess, 53.1ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 16 cars, 1 truck, 48.3ms
Speed: 1.0ms preprocess, 48.3ms inference, 2.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 16 cars, 1 truck, 44.4ms
Speed: 1.0ms preprocess, 44.4ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 19 cars, 2 trucks, 47.6ms
Speed: 1.5ms preprocess, 47.6ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 18 cars, 1 truck, 42.2ms
Speed: 0.0ms preprocess, 42.2ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 15 cars, 1 truck, 41.0ms
Speed: 0.0ms preprocess, 41.0ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 16 cars, 1 truck, 41.3ms
Speed: 0.0ms preprocess,